In [2]:
import pandas as pd

from sqlalchemy import create_engine

import pymysql
pymysql.install_as_MySQLdb()

# Change the following variables to match the season and week you need

In [3]:
season = 2018
week = 16

# Extraction of Data

In [20]:
#Extracting ESPN Projections


position_ids = [0,2,4,6] #qb=0 rb=2 wr=4 te=6
pages = [0,40]

dummy = pd.DataFrame()
for position in position_ids:
    for page in pages:
        url = f"http://games.espn.com/ffl/tools/projections?&slotCategoryId={position}&scoringPeriodId={week}&seasonId={season}&startIndex={page}"
        df = pd.read_html(url,header=1)
        df = df[1]
        if position == 0:
            pos = "QB"
        if position == 2:
            pos = "RB"
        if position == 4:
            pos = "WR"
        if position == 6:
            pos = "TE"
        df['POSITION'] = pos
        df['SEASON'] = season
        df['WEEK'] = week
        if position == 0 and page == 0:
            main = pd.concat([dummy,df])
        else:
            main = pd.concat([main,df])


In [5]:
#Extracting Site 2 
csv_file = f'Resources/Site_2_Projections/Site_2_W{week}_projections.csv'
four44_df = pd.read_csv(csv_file)

In [6]:
#Extracting Fantasy Sharks
csv_file = f'Resources/Fantasy_Sharks_Projections/W{week}_Projections.csv'

#Resources\Fantasy_Sharks_Projections\fantasysharks_W16_Projections.csv
sharks_df = pd.read_csv(csv_file)

# Transformation of Data

In [21]:
#transforming ESPN Data
df_clean = main['PLAYER, TEAM POS'].str.split(',',expand=True)
main['PLAYER'] = df_clean[0]
main = main.drop(columns=['PLAYER, TEAM POS'],axis=1)
main['FPTS'] = main['PTS'] + main['INT']
main.head()

,OPP,STATUS ET,C/A,YDS,TD,INT,RUSH,YDS.1,TD.1,REC,YDS.2,TD.2,PTS,POSITION,SEASON,WEEK,PLAYER,FPTS
0,@Sea,Sun 8:20,24.2/37.4,300.5,2.5,0.7,3.8,15.7,0.2,0.0,0.0,0,23.0,QB,2018,16,Patrick Mahomes,23.7
1,NYG,Sun 1:00,24.6/37.8,271.6,2.4,1.1,2.9,13.0,0.1,0.0,0.0,0,19.9,QB,2018,16,Andrew Luck,21.0
2,KC,Sun 8:20,18.1/27.5,217.6,2.3,0.5,4.3,23.9,0.1,0.0,0.0,0,19.8,QB,2018,16,Russell Wilson,20.3
3,@NYJ,Sun 1:00,23.4/37.7,284.8,1.8,0.3,3.0,16.7,0.1,0.0,0.0,0,19.8,QB,2018,16,Aaron Rodgers,20.1
4,TB,Sun 1:00,21.3/32.4,245.9,1.7,0.6,4.4,23.9,0.3,0.0,0.0,0,19.7,QB,2018,16,Dak Prescott,20.3


In [ ]:
#putting ESPN data into CSV
main.to_csv(f'resources/espn_projections/espn-week{week}-projections.csv')

In [ ]:
#transforming Site 2

# Applying Fanduel uniform scoring system to Fantasy Sharks raw predictions and creating a new column with the results
four44_df['Site 2'] = four44_df['Pass Yds'] * .04 + four44_df['Rush Yds'] * .1 + four44_df['Rec Yds'] * .1 + \
four44_df['Pass TD'] * 4 + four44_df['Rush TD'] * 6 + four44_df['Rec TD'] * 6 + four44_df['Rec'] * .5 + four44_df['INT'] * -1 \
+ four44_df['Fum'] * -2

#dropping unneeded columns
four44_df = four44_df.drop(['PID','Pos','Team','Opp','aFPA','aFPA Rk','FFPts','Comp','Pass Att','Pass Yds','Pass TD','INT',
'Rush Att','Rush Yds','Rush TD','Rec','Rec Yds','Rec TD','Pa1D','Ru1D','Rec1D','Fum','XP','FG','Grade'], axis=1)

# Rename player column to "Name"
four44_df.rename(columns = {"Player": "Name"}, inplace = True)

In [ ]:
#transforming Fantasy Sharks

# Splitting the name into 2 seperate columns
sharks_df[['Last Name','First Name']] = sharks_df['Name'].str.split(', ',expand=True)

# Recombine names into one column in the order of First Last
sharks_df['Name'] = sharks_df['First Name'].str.cat(sharks_df['Last Name'],sep=" ")

# Applying Fanduel uniform scoring system to Fantasy Sharks raw predictions and creating a new column with the results
sharks_df['Fantasy Sharks'] = sharks_df['Yards'] * .04 + sharks_df['Yards.1'] * .1 + sharks_df['Yards.2'] * .1 + \
sharks_df['TD'] * 4 + sharks_df['TD.1'] * 6 + sharks_df['TD.2'] * 6 + sharks_df['Rec'] * .5 + sharks_df['Int'] * -1 

#Removing unneeded columns
sharks_df = sharks_df.drop(['ID', 'Rank', 'First Name', 'Last Name', 'Comp', 'Yards', 'TD', 'TD.1', 
'TD.2', 'Int', 'Att', 'Yards.1', 'Yards.2', 'Rec', 'Fantasy Points'], axis=1)


In [ ]:
#combining cleaned DataFrames into one DataFrame

combined_df = sharks_df.merge(four44_df, on=['Name'], sort=False)

combined_df.round({'Fantasy Sharks': 1, 'Site 2': 1})


pd.options.display.float_format = '{:,.2f}'.format

combined_df = combined_df[['Season', 'Week', 'Name', 'Pos', 'Team', 'Opp', 'Fantasy Sharks', 'Site 2']]

combined_df.head()